<a href="https://colab.research.google.com/github/pszemraj/ai-msgbot/blob/main/colab-notebooks/%5BGPT2_dailydialogue%5D_aitextgen_text_generation_%2B_training_on_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  aitextgen — Train a GPT-2 (or GPT Neo) Text-Generating Model w/ GPU

by [Max Woolf](https://minimaxir.com)

*Last updated: May 16th, 2021 (aitextgen v0.5.2)*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Colaboratory** using `aitextgen`!

For more about `aitextgen`, you can visit [this GitHub repository](https://github.com/minimaxir/aitextgen) or [read the documentation](https://docs.aitextgen.io/).


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Run the cells below:


In [1]:
!nvidia-smi

Mon Oct 18 08:19:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    49W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## formatting

In [2]:
from IPython.display import HTML, display
# colab formatting
def set_css():
    display(
        HTML(
            """
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  """
        )
    )

get_ipython().events.register("pre_run_cell", set_css)

# setup

In [3]:
# update torch in case using a A100 GPU
!pip3 install torch==1.9.1+cu111 -f -q https://download.pytorch.org/whl/torch_stable.html

!pip3 install cudatoolkit

Looking in links: -q
     |████████████████████████████████| 227 kB 508 kB/s 
  ERROR: Cannot unpack file /tmp/pip-unpack-6qk9i48m/torch_stable.html (downloaded from /tmp/pip-req-build-5vzjx2vt, content-type: text/html); cannot detect archive format
ERROR: Cannot determine archive format of /tmp/pip-req-build-5vzjx2vt
ERROR: Could not find a version that satisfies the requirement cudatoolkit (from versions: none)
ERROR: No matching distribution found for cudatoolkit


In [4]:
!pip install -q aitextgen

import logging

logging.basicConfig(
    format="%(asctime)s — %(levelname)s — %(name)s — %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)

from aitextgen import aitextgen
from aitextgen.colab import mount_gdrive, copy_file_from_gdrive

In [5]:
mount_gdrive()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## GPU

Colaboratory uses a Nvidia P4, an Nvidia T4, an Nvidia P100, or an Nvidia V100. For finetuning GPT-2 124M, any of these GPUs will be fine, but for text generation, a T4 or a P100 is ideal since they have more VRAM. **If you receive a T4 or a V100 GPU, you can enable `fp16=True` during training for faster/more memory efficient training.**

You can verify which GPU is active by running the cell below. If you want to try for a different GPU, go to **Runtime -> Factory Reset Runtime**.

In [6]:
!nvidia-smi

Mon Oct 18 08:19:23 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    49W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Loading GPT-2 or GPT Neo

If you're retraining a model on new text, you need to download and load the GPT-2 model into the GPU. 

There are several sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M` (default): the "medium" model, 1.5GB on disk.
* `774M` (default): the "large" model, 3GB on disk.

You can also finetune a GPT Neo model instead, which is more suitable for longer texts and the base model has more recent data:

* `125M`: Analogous to the GPT-2 124M model.
* `350M`: Analogous to the GPT-2 355M model

The next cell downloads the model and saves it in the Colaboratory VM. If the model has already been downloaded, running this cell will reload it.

In [7]:
model_size = "355M" #@param ["355M", "774M"]
load_from_folder = True #@param {type:"boolean"}
load_folder_dir = "/content/drive/MyDrive/Programming/AI_peter/gpt2_dailydialogue_gpu_355M" #@param {type:"string"}


In [8]:
if load_from_folder:
    ai = aitextgen(model_folder=load_folder_dir, to_gpu=True,
                   gradient_checkpointing=True)
else:
    ai = aitextgen(tf_gpt2=model_size, to_gpu=True,
                gradient_checkpointing=True)
# ai = aitextgen(tf_gpt2="124M", to_gpu=True)
# https://huggingface.co/distilgpt2
# Comment out the above line and uncomment the below line to use GPT Neo instead.
# model_size
# ai = aitextgen(model='gpt2-medium', 
#                to_gpu=True, 
#                gradient_checkpointing=True)

10/18/2021 08:19:24 — INFO — aitextgen — Loading model from provided weights and config in //content/drive/MyDrive/Programming/AI_peter/gpt2_dailydialogue_gpu_355M.
/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
10/18/2021 08:19:28 — INFO — aitextgen — GPT2 loaded with 354M parameters.
10/18/2021 08:19:28 — INFO — aitextgen — Gradient checkpointing enabled for model training.
10/18/2021 08:19:28 — INFO — aitextgen — Using the default GPT-2 Tokenizer.


# load training data

In [9]:
dl_link = "https://www.dropbox.com/s/zngfhi9zw438fxk/daily_dialogue_augment.txt?dl=1" #@param {type:"string"}


In [10]:
# download test image
from urllib import request
from os.path import join
import os
vm_wd = os.getcwd()
local_name = join(vm_wd, "dailydialogue.txt")
request.urlretrieve(dl_link, local_name)


('/content/dailydialogue.txt', <http.client.HTTPMessage at 0x7f3e3c3361d0>)

In [11]:
# adjust names 


with open(local_name, 'r', encoding='utf-8', errors='ignore') as fi:
    orig_lines = fi.readlines()

from tqdm.auto import tqdm

upd_lines = []

for line in tqdm(orig_lines, total=len(orig_lines), desc="replacing speaker names"):
    fixline = line.replace("speaker a", "john smith")
    fixline = fixline.replace("speaker b", "nancy sellers")
    upd_lines.append(fixline)

local_namev2 = join(vm_wd, "V2dailydialogue.txt")

with open(local_namev2, 'w', encoding='utf-8', errors='ignore') as fo:
    fo.writelines(upd_lines)


replacing speaker names:   0%|          | 0/216768 [00:00<?, ?it/s]

In [12]:
import pprint as pp

pp.pprint(upd_lines[:10])

['john smith:\n',
 'say, jim, how about going for a few beers after dinner?\n',
 '\n',
 'nancy sellers:\n',
 'you know that is tempting but is really not good for our fitness.\n',
 '\n',
 'john smith:\n',
 'what do you mean? it will help us to relax.\n',
 '\n',
 'nancy sellers:\n']


In [13]:
file_name = local_namev2

If your text file is large (>10MB), it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

Additionally, you may want to consider [compressing the dataset to a cache first](https://docs.aitextgen.io/dataset/) on your local computer, then uploading the resulting `dataset_cache.tar.gz` and setting the `file_name`in the previous cell to that.

In [14]:
# copy_file_from_gdrive(local_name)

# Train / Finetune GPT-2

The next cell will start the actual finetuning of GPT-2 in aitextgen. It runs for `num_steps`, and a progress bar will appear to show training progress, current loss (the lower the better the model), and average loss (to give a sense on loss trajectory).

The model will be saved every `save_every` steps in `trained_model` by default, and when training completes. If you mounted your Google Drive, the model will _also_ be saved there in a unique folder.

The training might time out after 4ish hours; if you did not mount to Google Drive, make sure you end training and save the results so you don't lose them! (if this happens frequently, you may want to consider using [Colab Pro](https://colab.research.google.com/signup))

Important parameters for `train()`:

- **`line_by_line`**: Set this to `True` if the input text file is a single-column CSV, with one record per row. aitextgen will automatically process it optimally.
- **`from_cache`**: If you compressed your dataset locally (as noted in the previous section) and are using that cache file, set this to `True`.
- **`num_steps`**: Number of steps to train the model for.
- **`generate_every`**: Interval of steps to generate example text from the model; good for qualitatively validating training.
- **`save_every`**: Interval of steps to save the model: the model will be saved in the VM to `/trained_model`.
- **`save_gdrive`**: Set this to `True` to copy the model to a unique folder in your Google Drive, if you have mounted it in the earlier cells
- **`fp16`**: Enables half-precision training for faster/more memory-efficient training. Only works on a T4 or V100 GPU.

Here are other important parameters for `train()` that are useful but you likely do not need to change.

- **`learning_rate`**: Learning rate of the model training.
- **`batch_size`**: Batch size of the model training; setting it too high will cause the GPU to go OOM. (if using `fp16`, you can increase the batch size more safely)

In [15]:
import gc, os
from os.path import join

base_dir = "/content/drive/MyDrive/Programming/AI_peter"
temp_gpu_path = join(base_dir, "gp2_dailydialogue_{}".format(model_size))
os.makedirs(temp_gpu_path, exist_ok=True)
gc.collect()

0

In [ ]:
# DO NOT USE WARMUP STEPS

ai.train(file_name,
         output_dir=temp_gpu_path,
         line_by_line=False,
         from_cache=False,
         num_steps=75000, # takes about 5 hours on 16 gb GPU for 75000
         generate_every=1500,
         max_grad_norm=0.8,
         save_every=1000,
         gradient_accumulation_steps=1,
         save_gdrive=False,
         learning_rate=1e-3,
         fp16=True,
         batch_size=1, 
         fp16_opt_level="O1",
         )

10/18/2021 08:19:36 — INFO — aitextgen — Loading text from /content/V2dailydialogue.txt with generation length of 1024.


  0%|          | 0/216768 [00:00<?, ?it/s]

10/18/2021 08:19:36 — INFO — aitextgen.TokenDataset — Encoding 216,768 sets of tokens from /content/V2dailydialogue.txt.
10/18/2021 08:19:39 — WARNING — aitextgen — pytorch_model.bin already exists in //content/drive/MyDrive/Programming/AI_peter/gp2_dailydialogue_355M and will be overwritten!
10/18/2021 08:19:39 — INFO — pytorch_lightning.trainer.connectors.accelerator_connector — Using native 16bit precision.
10/18/2021 08:19:39 — INFO — pytorch_lightning.utilities.distributed — GPU available: True, used: True
10/18/2021 08:19:39 — INFO — pytorch_lightning.utilities.distributed — TPU available: False, using: 0 TPU cores
10/18/2021 08:19:39 — INFO — pytorch_lightning.utilities.distributed — IPU available: False, using: 0 IPUs
10/18/2021 08:19:39 — INFO — pytorch_lightning.accelerators.gpu — LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


  0%|          | 0/75000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/plugins/precision/precision_plugin.py:141: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  torch.nn.utils.clip_grad_norm_(parameters, clip_val)


1,000 steps reached: saving model to //content/drive/MyDrive/Programming/AI_peter/gp2_dailydialogue_355M
1,500 steps reached: generating sample texts.
. i think i'll be going to go to work right away.

john smith:
excuse me, sir. could i have your name?

nancy sellers:
here's your card, sir.

john smith:
thank you. what's your bill for?

nancy sellers:
do you need your account number?

john smith:
yes, here's your bill.

nancy sellers:
here's your bill, i know your bill here.

john smith:
thank you.

nancy sellers:
hello, sir. can you make a reservation?

john smith:
sure, how dare you?

nancy sellers:
i think i will be right back.

john smith:
ok, here is a bill.

nancy sellers:
would you like to make a reservation for friday?

john smith:
sure. we have many plans for friday is a birthday for you.

nancy sellers:
if that's the case, we'll pick you up at the last moment.

john smith:
so, i'll have a good
2,000 steps reached: saving model to //content/drive/MyDrive/Programming/AI_peter/

In [ ]:
save_path = "/content/drive/MyDrive/Programming/AI_peter/gpt2_dailydialogue_gpu_{}".format(model_size)

In [ ]:
import os
os.makedirs(save_path, exist_ok=True)
ai.save(save_path)


You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.


# Use a Train Model for Generation

If you already had a trained model from this notebook, running the next cell will copy the `pytorch_model.bin` and the `config.json`file from the specified folder in Google Drive into the Colaboratory VM. (If no `from_folder` is specified, it assumes the two files are located at the root level of your Google Drive)

In [ ]:
!nvidia-smi

In [ ]:
mount_gdrive()


In [ ]:
# best model thus far @ 1.3B parameters and tuned for 50k steps
# from_folder = "/content/drive/MyDrive/Programming/AI_peter/GPT-Neo-1B-V1"

from_folder = save_path

if len(from_folder) > 2:

    for file in ["pytorch_model.bin", "config.json"]:
        if from_folder:
            copy_file_from_gdrive(file, from_folder)
        else:
            copy_file_from_gdrive(file)

    ai = aitextgen(model_folder=from_folder, to_gpu=True)
else:
    ai = aitextgen(model_folder=".", to_gpu=True)


## Generate Text From The Trained Model


`generate()` without any parameters generates a single text from the loaded model to the console.

In [ ]:
ai.generate(n=3, max_length=256, 
            temperature=1.0, top_p=0.9)

In [ ]:
ai.generate(prompt="give me a good pickup line!", temperature=1,
            min_length=10, batch_size =20)

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = ai.generate_one()`

You can also pass in a `prompt` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `n`. You can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 50 for `batch_size` to avoid going OOM).

Other optional-but-helpful parameters for `ai.generate()` and friends:

*  **`min length`**: The minimum length of the generated text: if the text is shorter than this value after cleanup, aitextgen will generate another one.
*  **`max_length`**: Number of tokens to generate (default 256, you can generate up to 1024 tokens with GPT-2 and 2048 with GPT Neo)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)

In [ ]:
ai.generate(
    n=3, batch_size=25, prompt="i just", max_length=256, 
    temperature=1.0, top_p=0.9
)

For bulk generation, you can generate a large amount of texts to a file and sort out the samples locally on your computer. The next cell will generate `num_files` files, each with `n` texts and whatever other parameters you would pass to `generate()`. The files can then be downloaded from the Files sidebar!

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
save_loc = "/content/drive/MyDrive/Programming/AI_peter/output_files" #@param {type:"string"}


In [ ]:
p_list = [["nancy powell:"+"\n", 
           "how are you doing?"+"\n", "\n", 
           "john smith:" + "\n"], 
           ["nancy powell:"+"\n", "why is "],
           ["can you help me with my homework?"+"\n", "\n", 
            "john smith:" + "\n"],
           ["john smith:" + "\n"],
           ["Hey I’m meeting the astrophysics professor via zoom after school any tips?"+"\n",
            "\n", "john smith:" + "\n"], 
]


prompts = ["".join(line) for line in p_list]

In [ ]:
from datetime import datetime
import pprint as pp

ds_date_time = datetime.now().strftime("%m.%d.%Y")

base_header = "gpt-dailydialogue-textgen-{}".format(ds_date_time)
prompt_IDs = [base_header + "_file-{}.txt".format(i+1) for i in range(5, len(prompts)+11)]

prompt_mng = {}
for pid, text in zip(prompt_IDs, prompts):
    prompt_mng[pid] = text
pp.pprint(prompt_mng)

In [ ]:
from os.path import join

for pfile, my_prompt in prompt_mng.items():
    ai.generate_to_file(
        n=50,
        batch_size=5,
        prompt=my_prompt,
        max_length=512,
        temperature=0.8,
        top_p=0.9,
        destination_path=join(save_loc, pfile)
    )
